# GENERATIVE TEXT TO IMAGE SYNTHESIS PART 1

## DOWNLOADING THE DATA

In [1]:
#installing the gdown
!pip install -U gdown 

#creating the Flowers Data Directory
!mkdir -p Data/Flowers/ 

#downloading the Flowers Data
!wget https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz
    
#moving the Flowers Data
!mv 102flowers.tgz Data/Flowers/

#downloading Flowers Caption
!gdown 0B0ywwgffWnLLMl9uOU91MV80cVU

#moving Flowers Caption
!mv flowers_icml.tar.gz Data/Flowers/

#downloading Flowers metadata for sepreating train images
!gdown 0B0ywwgffWnLLcms2WWJQRFNSWXM

#moving Flowers metadata
!mv cvpr2016_flowers.tar.gz Data/Flowers/



















--2023-03-30 16:06:22--  https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz

Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2

Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.

HTTP request sent, awaiting response... 301 Moved Permanently

Location: https://thor.robots.ox.ac.uk/datasets/flowers-102/102flowers.tgz [following]

--2023-03-30 16:06:23--  https://thor.robots.ox.ac.uk/datasets/flowers-102/102flowers.tgz

Resolving thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)... 129.67.95.98

Connecting to thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)|129.67.95.98|:443... connected.

HTTP request sent, awaiting response... 200 OK

Length: 344862509 (329M) [application/octet-stream]

Saving to: ‘102flowers.tgz’



102flowers.tgz      100%[===================>] 328.89M  21.2MB/s    in 17s     



2023-03-30 16:06:41 (18.8 MB/s) - ‘102flowers.tgz’ saved [344862509/344862509]



Downloading...

From (uriginal): htt

## IMPORTING LIBRARIES

In [2]:

import tarfile#FOR EXTRACTING TAR FILES

import theano
import theano.tensor as tensor #FOR HANDLING MATHEMATICAL EXPRESSION OF MULTI DIMENSION ARRAY'S
import _pickle as pkl#STORING THE PREPROCESSED DATA IN PICKLE FILES
import numpy#MATHEMATICAL LIBRARY
import copy#TO CREATE OBJECT COPIES
import nltk# NATURAL LANGUAGE MODULE
from collections import OrderedDict, defaultdict#TO STORE DATA IN COLLECTIONS FORM
from scipy.linalg import norm#FOR SCIENTIFIC CALCULATION
from nltk.tokenize import word_tokenize#FOR WORD TOKENIZATION
import os#FOR OS RELATED FUNCTIONS
import traceback
import pickle
import random
import numpy as np
from os.path import join

## EXTRACTING THE DATA FILES

In [3]:
for tar_file in os.listdir('Data/Flowers/'):
    
    tmp=os.system('mkdir -p Data/Flowers/'+tar_file.split('.')[0]+'/')
    
    print('Extracting Data/Flowers/'+tar_file)#Extracting each Downloaded comressed files(.tar)
    
    tarfile.open('Data/Flowers/'+tar_file).extractall(path='Data/Flowers/'+tar_file.split('.')[0]+'/')

Extracting Data/Flowers/flowers_icml.tar.gz

Extracting Data/Flowers/cvpr2016_flowers.tar.gz

Extracting Data/Flowers/102flowers.tgz


## DOWNLOADING THE SKIPTHOUGH VECTORS MODEL

In [4]:
#downloading pre-trained skip thought model
!mkdir -p Data/Flowers/skipthoughts/
!wget http://www.cs.toronto.edu/~rkiros/models/dictionary.txt
!mv dictionary.txt Data/Flowers/skipthoughts/
!wget http://www.cs.toronto.edu/~rkiros/models/utable.npy
!mv utable.npy Data/Flowers/skipthoughts/
!wget http://www.cs.toronto.edu/~rkiros/models/btable.npy
!mv btable.npy  Data/Flowers/skipthoughts/
!wget http://www.cs.toronto.edu/~rkiros/models/uni_skip.npz
!mv uni_skip.npz Data/Flowers/skipthoughts/
!wget http://www.cs.toronto.edu/~rkiros/models/uni_skip.npz.pkl
!mv uni_skip.npz.pkl Data/Flowers/skipthoughts/
!wget http://www.cs.toronto.edu/~rkiros/models/bi_skip.npz
!mv bi_skip.npz Data/Flowers/skipthoughts/
!wget http://www.cs.toronto.edu/~rkiros/models/bi_skip.npz.pkl
!mv bi_skip.npz.pkl Data/Flowers/skipthoughts/


--2023-03-30 16:08:06--  http://www.cs.toronto.edu/~rkiros/models/dictionary.txt

Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30

Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:80... connected.

HTTP request sent, awaiting response... 200 OK

Length: 7996547 (7.6M) [text/plain]

Saving to: ‘dictionary.txt’



dictionary.txt      100%[===================>]   7.63M  11.2MB/s    in 0.7s    



2023-03-30 16:08:07 (11.2 MB/s) - ‘dictionary.txt’ saved [7996547/7996547]



--2023-03-30 16:08:09--  http://www.cs.toronto.edu/~rkiros/models/utable.npy

Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30

Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:80... connected.

HTTP request sent, awaiting response... 200 OK

Length: 2342138474 (2.2G)

Saving to: ‘utable.npy’



utable.npy          100%[===================>]   2.18G  48.0MB/s    in 53s     



2023-03-30 16:09:03 (41.8 MB/s) - ‘utable.npy’ saved [2342138474/2342138

## FUNCTION FOR USING THE SKIP THOUGHT MODEL TO CONVERT CAPTION INTO SKIP THOUGH VECTORS

In [5]:

#SETTING DOWNLOADED MODEL PATH

#-----------------------------------------------------------------------------#
path_to_models = 'Data/Flowers/skipthoughts/'
path_to_tables = 'Data/Flowers/skipthoughts/'


path_to_umodel = path_to_models + 'uni_skip.npz'
path_to_bmodel = path_to_models + 'bi_skip.npz'
#-----------------------------------------------------------------------------#

# FUNCTION TO LOAD SKIP THOUGHTS MODEL
def load_model():
    
    # Load model PARAMS
    print('Loading model parameters...')
    
#     READING THE MODEL FILE
    with open('%s.pkl'%path_to_umodel, 'rb') as f:
        uoptions = pkl.load(f)
    with open('%s.pkl'%path_to_bmodel, 'rb') as f:
        boptions = pkl.load(f)

    # Load parameters
    uparams = init_params(uoptions)
    uparams = load_params(path_to_umodel, uparams)
    utparams = init_tparams(uparams)
    bparams = init_params_bi(boptions)
    bparams = load_params(path_to_bmodel, bparams)
    btparams = init_tparams(bparams)

    # Extractor functions FOR THE MODEL FILE
    print('Compiling encoders...')
    embedding, x_mask, ctxw2v = build_encoder(utparams, uoptions)
    f_w2v = theano.function([embedding, x_mask], ctxw2v, name='f_w2v')
    embedding, x_mask, ctxw2v = build_encoder_bi(btparams, boptions)
    f_w2v2 = theano.function([embedding, x_mask], ctxw2v, name='f_w2v2')

    # Tables FOR SKIP THOUGHT
    print('Loading tables...')
    utable, btable = load_tables()

    # Store everything we need in a dictionary
    print('Packing up...')
    model = {}
    model['uoptions'] = uoptions
    model['boptions'] = boptions
    model['utable'] = utable
    model['btable'] = btable
    model['f_w2v'] = f_w2v
    model['f_w2v2'] = f_w2v2

    return model

# FUNCTION TO LOAD TABLE
def load_tables():
    words = []
    utable = numpy.load(path_to_tables+'utable.npy', encoding='bytes',allow_pickle=True)
    btable = numpy.load(path_to_tables+'btable.npy', encoding='bytes',allow_pickle=True)
    f = open(path_to_tables+'dictionary.txt', 'rb')
    for line in f:
        words.append(line.decode('utf-8').strip())
    f.close()
    utable = OrderedDict(zip(words, utable))
    btable = OrderedDict(zip(words, btable))
    return utable, btable

# Encode sentences in the list X. Each entry will return a vector
def encode(model, X, use_norm=True, verbose=True, batch_size=128, use_eos=False):
    
    # first, do preprocessing
    X = preprocess(X)

    # word dictionary and initialization
    d = defaultdict(lambda : 0)
    for w in model['utable'].keys():
        d[w] = 1
    ufeatures = numpy.zeros((len(X), model['uoptions']['dim']), dtype='float32')
    bfeatures = numpy.zeros((len(X), 2 * model['boptions']['dim']), dtype='float32')

    # find length of dictionary
    ds = defaultdict(list)
    captions = [s.split() for s in X]
    for i,s in enumerate(captions):
        ds[len(s)].append(i)

    # Get features. This encodes by length, in order to avoid wasting computation
    for k in ds.keys():
        if verbose:
            print(k)
        numbatches = len(ds[k]) / batch_size + 1
        for minibatch in range(int(numbatches)):
            caps = ds[k][int(minibatch)::int(numbatches)]

            if use_eos:
                uembedding = numpy.zeros((k+1, len(caps), model['uoptions']['dim_word']), dtype='float32')
                bembedding = numpy.zeros((k+1, len(caps), model['boptions']['dim_word']), dtype='float32')
            else:
                uembedding = numpy.zeros((k, len(caps), model['uoptions']['dim_word']), dtype='float32')
                bembedding = numpy.zeros((k, len(caps), model['boptions']['dim_word']), dtype='float32')
            for ind, c in enumerate(caps):
                caption = captions[c]
                for j in range(len(caption)):
                    if d[caption[j]] > 0:
                        uembedding[j,ind] = model['utable'][caption[j]]
                        bembedding[j,ind] = model['btable'][caption[j]]
                    else:
                        uembedding[j,ind] = model['utable']['UNK']
                        bembedding[j,ind] = model['btable']['UNK']
                if use_eos:
                    uembedding[-1,ind] = model['utable']['<eos>']
                    bembedding[-1,ind] = model['btable']['<eos>']
            if use_eos:
                uff = model['f_w2v'](uembedding, numpy.ones((len(caption)+1,len(caps)), dtype='float32'))
                bff = model['f_w2v2'](bembedding, numpy.ones((len(caption)+1,len(caps)), dtype='float32'))
            else:
                uff = model['f_w2v'](uembedding, numpy.ones((len(caption),len(caps)), dtype='float32'))
                bff = model['f_w2v2'](bembedding, numpy.ones((len(caption),len(caps)), dtype='float32'))
            if use_norm:
                for j in range(len(uff)):
                    uff[j] /= norm(uff[j])
                    bff[j] /= norm(bff[j])
            for ind, c in enumerate(caps):
                ufeatures[c] = uff[ind]
                bfeatures[c] = bff[ind]

    features = numpy.c_[ufeatures, bfeatures]
    return features

# FUNCTION TO PREPROCESS TEXT FOR ENCODERS
def preprocess(text):
    X = []
    sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
    for t in text:
        sents = sent_detector.tokenize(t)
        result = ''
        for s in sents:
            tokens = word_tokenize(s)
            result += ' ' + ' '.join(tokens)
        X.append(result)
    return X


# Function to Return the nearest neighbour sentences to query
# Input =>    text: list of sentences
#             vectors: the corresponding representations for text
#             query: a string to search
def nn(model, text, vectors, query, k=5):
    qf = encode(model, [query])
    qf /= norm(qf)
    scores = numpy.dot(qf, vectors.T).flatten()
    sorted_args = numpy.argsort(scores)[::-1]
    sentences = [text[a] for a in sorted_args[:k]]
    print('QUERY: ' + query)
    print('NEAREST: ')
    for i, s in enumerate(sentences):
        print(s, sorted_args[i])

# Function to Extract word features into a normalized matrix
def word_features(table):
    features = numpy.zeros((len(table), 620), dtype='float32')
    keys = table.keys()
    for i in range(len(table)):
        f = table[keys[i]]
        features[i] = f / norm(f)
    return features

# Function to Get the nearest neighbour words
def nn_words(table, wordvecs, query, k=10):
    keys = table.keys()
    qf = table[query]
    scores = numpy.dot(qf, wordvecs.T).flatten()
    sorted_args = numpy.argsort(scores)[::-1]
    words = [keys[a] for a in sorted_args[:k]]
    print('QUERY: ' + query)
    print('NEAREST: ')
    for i, w in enumerate(words):
        print(w)

# make prefix-appended name
def _p(pp, name):
    return '%s_%s'%(pp, name)

# initialize Theano shared variables according to the initial parameters
def init_tparams(params):
    tparams = OrderedDict()
    for kk, pp in params.items():
        tparams[kk] = theano.shared(params[kk], name=kk)
    return tparams

# load parameters
def load_params(path, params):
    pp = numpy.load(path)
    for kk, vv in params.items():
        if kk not in pp:
            warnings.warn('%s is not in the archive'%kk)
            continue
        params[kk] = pp[kk]
    return params


# layers: 'name': ('parameter initializer', 'feedforward')
layers = {'gru': ('param_init_gru', 'gru_layer')}

def get_layer(name):
    fns = layers[name]
    return (eval(fns[0]), eval(fns[1]))

# initialize all parameters needed for the encoder
def init_params(options):
    params = OrderedDict()

    # embedding
    params['Wemb'] = norm_weight(options['n_words_src'], options['dim_word'])

    # encoder: GRU
    params = get_layer(options['encoder'])[0](options, params, prefix='encoder',
                                              nin=options['dim_word'], dim=options['dim'])
    return params

# initialize all paramters needed for bidirectional encoder
def init_params_bi(options):
    params = OrderedDict()

    # embedding
    params['Wemb'] = norm_weight(options['n_words_src'], options['dim_word'])

    # encoder: GRU
    params = get_layer(options['encoder'])[0](options, params, prefix='encoder',
                                              nin=options['dim_word'], dim=options['dim'])
    params = get_layer(options['encoder'])[0](options, params, prefix='encoder_r',
                                              nin=options['dim_word'], dim=options['dim'])
    return params

# build an encoder, given pre-computed word embeddings
def build_encoder(tparams, options):
    # word embedding (source)
    embedding = tensor.tensor3('embedding', dtype='float32')
    x_mask = tensor.matrix('x_mask', dtype='float32')

    # encoder
    proj = get_layer(options['encoder'])[1](tparams, embedding, options,
                                            prefix='encoder',
                                            mask=x_mask)
    ctx = proj[0][-1]

    return embedding, x_mask, ctx

# build bidirectional encoder, given pre-computed word embeddings
def build_encoder_bi(tparams, options):
    # word embedding (source)
    embedding = tensor.tensor3('embedding', dtype='float32')
    embeddingr = embedding[::-1]
    x_mask = tensor.matrix('x_mask', dtype='float32')
    xr_mask = x_mask[::-1]

    # encoder
    proj = get_layer(options['encoder'])[1](tparams, embedding, options,
                                            prefix='encoder',
                                            mask=x_mask)
    projr = get_layer(options['encoder'])[1](tparams, embeddingr, options,
                                             prefix='encoder_r',
                                             mask=xr_mask)

    ctx = tensor.concatenate([proj[0][-1], projr[0][-1]], axis=1)

    return embedding, x_mask, ctx


# some utilities function
def ortho_weight(ndim):
    W = numpy.random.randn(ndim, ndim)
    u, s, v = numpy.linalg.svd(W)
    return u.astype('float32')


def norm_weight(nin,nout=None, scale=0.1, ortho=True):
    if nout == None:
        nout = nin
    if nout == nin and ortho:
        W = ortho_weight(nin)
    else:
        W = numpy.random.uniform(low=-scale, high=scale, size=(nin, nout))
    return W.astype('float32')

# parameter init for GRU
def param_init_gru(options, params, prefix='gru', nin=None, dim=None):
    if nin == None:
        nin = options['dim_proj']
    if dim == None:
        dim = options['dim_proj']
    W = numpy.concatenate([norm_weight(nin,dim),
                           norm_weight(nin,dim)], axis=1)
    params[_p(prefix,'W')] = W
    params[_p(prefix,'b')] = numpy.zeros((2 * dim,)).astype('float32')
    U = numpy.concatenate([ortho_weight(dim),
                           ortho_weight(dim)], axis=1)
    params[_p(prefix,'U')] = U

    Wx = norm_weight(nin, dim)
    params[_p(prefix,'Wx')] = Wx
    Ux = ortho_weight(dim)
    params[_p(prefix,'Ux')] = Ux
    params[_p(prefix,'bx')] = numpy.zeros((dim,)).astype('float32')

    return params

# Forward pass through GRU layer
def gru_layer(tparams, state_below, options, prefix='gru', mask=None, **kwargs):
    nsteps = state_below.shape[0]
    if state_below.ndim == 3:
        n_samples = state_below.shape[1]
    else:
        n_samples = 1

    dim = tparams[_p(prefix,'Ux')].shape[1]

    if mask == None:
        mask = tensor.alloc(1., state_below.shape[0], 1)

    def _slice(_x, n, dim):
        if _x.ndim == 3:
            return _x[:, :, n*dim:(n+1)*dim]
        return _x[:, n*dim:(n+1)*dim]

    state_below_ = tensor.dot(state_below, tparams[_p(prefix, 'W')]) + tparams[_p(prefix, 'b')]
    state_belowx = tensor.dot(state_below, tparams[_p(prefix, 'Wx')]) + tparams[_p(prefix, 'bx')]
    U = tparams[_p(prefix, 'U')]
    Ux = tparams[_p(prefix, 'Ux')]

    def _step_slice(m_, x_, xx_, h_, U, Ux):
        preact = tensor.dot(h_, U)
        preact += x_

        r = tensor.nnet.sigmoid(_slice(preact, 0, dim))
        u = tensor.nnet.sigmoid(_slice(preact, 1, dim))

        preactx = tensor.dot(h_, Ux)
        preactx = preactx * r
        preactx = preactx + xx_

        h = tensor.tanh(preactx)

        h = u * h_ + (1. - u) * h
        h = m_[:,None] * h + (1. - m_)[:,None] * h_

        return h

    seqs = [mask, state_below_, state_belowx]
    _step = _step_slice

    rval, updates = theano.scan(_step,
                                sequences=seqs,
                                outputs_info = [tensor.alloc(0., n_samples, dim)],
                                non_sequences = [tparams[_p(prefix, 'U')],
                                                 tparams[_p(prefix, 'Ux')]],
                                name=_p(prefix, '_layers'),
                                n_steps=nsteps,
                                profile=False,
                                strict=True)
    rval = [rval]
    return rval


## FUNCTION FOR DATA PREPROCESSING

In [6]:

# fucntion for one hotencoding of targets
def get_one_hot_targets(target_file_path):
    target = []
    one_hot_targets = []
    n_target = 0
    try :
        with open(target_file_path) as f :
            target = f.readlines()
            target = [t.strip('\n') for t in target]
            n_target = len(target)
    except IOError :
        print('Could not load the labels.txt file in the dataset. A '
              'dataset folder is expected in the "Data/Flowers/" '
              'directory with the name that has been passed as an '
              'argument to this method. This directory should contain a '
              'file called labels.txt which contains a list of labels and '
              'corresponding folders for the labels with the same name as '
              'the labels.')
        traceback.print_stack()

    lbl_idxs = np.arange(n_target)
    one_hot_targets = np.zeros((n_target, n_target))
    one_hot_targets[np.arange(n_target), lbl_idxs] = 1

    return target, one_hot_targets, n_target


# Encodes a string label into one-hot encoding
def one_hot_encode_str_lbl(lbl, target, one_hot_targets):
        idx = target.index(lbl)
        return one_hot_targets[idx]

# save the flowers image captions as vector
def save_caption_vectors_flowers(data_dir, dt_range=(1, 103)) :
    import time

    img_dir = join(data_dir, '102flowers/jpg')
    all_caps_dir = join(data_dir, 'all_captions.txt')
    target_file_path = os.path.join(data_dir,'cvpr2016_flowers/allclasses.txt')
    caption_dir = join(data_dir, 'cvpr2016_flowers/text_c10')
    image_files = [f for f in os.listdir(img_dir) if 'jpg' in f]
    print(image_files[300 :400])
    image_captions = {}
    image_classes = {}
    class_dirs = []
    class_names = []
    img_ids = []

    target, one_hot_targets, n_target = get_one_hot_targets(target_file_path)

    for i in range(dt_range[0], dt_range[1]) :
        class_dir_name = 'class_%.5d' % (i)
        class_dir = join(caption_dir, class_dir_name)
        class_names.append(class_dir_name)
        class_dirs.append(class_dir)
        onlyimgfiles = [f[0 :11] + ".jpg" for f in os.listdir(class_dir)
                                    if 'txt' in f]
        for img_file in onlyimgfiles:
            image_classes[img_file] = None

        for img_file in onlyimgfiles:
            image_captions[img_file] = []

    for class_dir, class_name in zip(class_dirs, class_names) :
        caption_files = [f for f in os.listdir(class_dir) if 'txt' in f]
        for i, cap_file in enumerate(caption_files) :
            if i%50 == 0:
                print(str(i) + ' captions extracted from' + str(class_dir))
            with open(join(class_dir, cap_file)) as f :
                str_captions = f.read()
                captions = str_captions.split('\n')
            img_file = cap_file[0 :11] + ".jpg"

            # 5 captions per image
            image_captions[img_file] += [cap for cap in captions if len(cap) > 0][0 :5]
            image_classes[img_file] = one_hot_encode_str_lbl(class_name,
                                                             target,
                                                             one_hot_targets)

    model = load_model()
    encoded_captions = {}
    for i, img in enumerate(image_captions) :
        st = time.time()
        encoded_captions[img] = encode(model, image_captions[img])
        if i%20 == 0:
            print(i, len(image_captions), img)
            print("Seconds", time.time() - st)

    img_ids = list(image_captions.keys())

    random.shuffle(img_ids)
    n_train_instances = int(len(img_ids) * 0.9)
    tr_image_ids = img_ids[0 :n_train_instances]
    val_image_ids = img_ids[n_train_instances : -1]
    
    !mkdir -p Data/Flowers/pickles
    pickle.dump(image_captions,
                open(os.path.join(data_dir,'pickles/flowers_caps.pkl'), "wb"))

    pickle.dump(tr_image_ids,
                open(os.path.join(data_dir,'pickles/train_ids.pkl'), "wb"))
    pickle.dump(val_image_ids,
                open(os.path.join(data_dir,'pickles/val_ids.pkl'), "wb"))

    ec_pkl_path = (join( data_dir,'pickles/flower_tv.pkl'))
    pickle.dump(encoded_captions, open(ec_pkl_path, "wb"))

    fc_pkl_path = (join(data_dir,'pickles/flower_tc.pkl'))
    pickle.dump(image_classes, open(fc_pkl_path, "wb"))

dataset = 'flowers'
    
if dataset == 'flowers':
    save_caption_vectors_flowers('Data/Flowers')
else:
    print('Preprocessor for this dataset is not available.')




['image_02456.jpg', 'image_04222.jpg', 'image_04660.jpg', 'image_02379.jpg', 'image_05078.jpg', 'image_00996.jpg', 'image_02191.jpg', 'image_01090.jpg', 'image_05698.jpg', 'image_07697.jpg', 'image_00089.jpg', 'image_04455.jpg', 'image_03428.jpg', 'image_06197.jpg', 'image_04442.jpg', 'image_06730.jpg', 'image_05641.jpg', 'image_03387.jpg', 'image_01989.jpg', 'image_03689.jpg', 'image_05586.jpg', 'image_00524.jpg', 'image_05307.jpg', 'image_05847.jpg', 'image_00787.jpg', 'image_03029.jpg', 'image_04374.jpg', 'image_00450.jpg', 'image_01134.jpg', 'image_01397.jpg', 'image_07355.jpg', 'image_05925.jpg', 'image_08033.jpg', 'image_02038.jpg', 'image_01730.jpg', 'image_01445.jpg', 'image_05933.jpg', 'image_03782.jpg', 'image_01352.jpg', 'image_01822.jpg', 'image_03452.jpg', 'image_02661.jpg', 'image_05219.jpg', 'image_00289.jpg', 'image_00570.jpg', 'image_04149.jpg', 'image_01508.jpg', 'image_06491.jpg', 'image_01574.jpg', 'image_08104.jpg', 'image_00435.jpg', 'image_06871.jpg', 'image_0320

# 

# ------------------------------------------------THE END------------------------------------------------